In [60]:
import os
import requests
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd
import time
import scrapy
from scrapy_playwright.page import PageMethod
from bs4 import BeautifulSoup
import nest_asyncio
import glob
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process
import re


In [61]:
Fresh_Leads_formatted = pd.read_parquet("staging/Fresh_Leads_formatted.parquet", engine="fastparquet")
Fresh_Leads_formatted.head()

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,None,None,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,None,None,Gov List,"[ACRA, Google Searches]",No,None
1,None,None,2025-11-10,NUU&NII PTE. LTD.,NUU&NII,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,None,None,Gov List,"[ACRA, Google Searches]",No,None
2,None,None,2025-11-10,CLOUD FRONTIER ASIA PTE. LTD.,CLOUD FRONTIER ASIA,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,None,None,Gov List,"[ACRA, Google Searches]",No,None
3,None,None,2025-11-10,GOLDEN GARLAND COMMUNICATION PTE. LTD.,GOLDEN GARLAND COMMUNICATION,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,None,None,Gov List,"[ACRA, Google Searches]",No,None
4,None,None,2025-11-10,XINYUAN SHOES PTE. LTD.,XINYUAN SHOES,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,None,None,Gov List,"[ACRA, Google Searches]",No,None


In [62]:
epos_backend_df = pd.read_csv(
    "./Epos_Backend/organizations_export.csv",
    on_bad_lines="skip"  # skips rows with too many or too few fields
)

epos_backend_df = epos_backend_df.loc[
    epos_backend_df["status"] == "Active",
    ["organization_name", "status"]
]

epos_backend_df


,organization_name,status
0,FRESH VALUE PTE. LTD.,Active
1,Ace Mobile,Active
2,Star Cafe Singapore Pte Ltd,Active
3,SG LOCAL FOOD PTE. LTD.,Active
4,Sree Vari Pte Ltd,Active
...,...,...
2338,ANGEL SUPERMART PTE LTD,Active
2339,DF ACADEMY PTE LTD,Active
2340,POH BROS. GOLF MANAGEMENT PTE LTD,Active
2341,THAT AQUARIUM,Active


In [63]:

# Initialize "Current ePOS Client ?" column with "No" for all rows
Fresh_Leads_formatted['Current ePOS Client ?'] = 'No'

# Clean and prepare the data for matching
# Convert to string and strip, but keep all rows (including NaN)
Fresh_Leads_formatted['ACRA REGISTERED NAME_cleaned'] = Fresh_Leads_formatted['ACRA REGISTERED NAME'].astype(str).str.strip()
epos_backend_names = epos_backend_df['organization_name'].dropna().astype(str).str.strip().tolist()

# Set similarity threshold (100% match - exact match only)
THRESHOLD = 100

# Track matches
matches_found = []
matched_indices = []

# Get rows with valid (non-null, non-nan) names
valid_name_mask = (
    Fresh_Leads_formatted['ACRA REGISTERED NAME'].notna() & 
    (Fresh_Leads_formatted['ACRA REGISTERED NAME_cleaned'] != 'nan') &
    (Fresh_Leads_formatted['ACRA REGISTERED NAME_cleaned'] != '')
)

valid_rows = Fresh_Leads_formatted[valid_name_mask]
print(f"Checking {len(valid_rows)} names against {len(epos_backend_names)} ePOS backend organizations...\n")
print("=" * 80)

# For each row with a valid name, check if it exists in ePOS backend
for idx, row in valid_rows.iterrows():
    name = row['ACRA REGISTERED NAME_cleaned']
    
    # Find the best match using fuzzy matching
    best_match = process.extractOne(
        name, 
        epos_backend_names,
        scorer=fuzz.token_sort_ratio
    )
    
    if best_match and best_match[1] >= THRESHOLD:
        matches_found.append({
            'Fresh Lead Name': name,
            'Matched ePOS Name': best_match[0],
            'Similarity Score': best_match[1]
        })
        matched_indices.append(idx)  # Store the original index
        # Update the dataframe: set "Current ePOS Client ?" to "Yes" for this row
        Fresh_Leads_formatted.at[idx, 'Current ePOS Client ?'] = 'Yes'
        print(f"yes there's exist!")
        print(f"  Fresh Lead: {name}")
        print(f"  Matched with: {best_match[0]}")
        print(f"  Similarity: {best_match[1]}%")
        print("-" * 80)

# Drop the temporary cleaned column
Fresh_Leads_formatted = Fresh_Leads_formatted.drop(columns=['ACRA REGISTERED NAME_cleaned'])

print(f"\n\nTotal matches found: {len(matches_found)} out of {len(valid_rows)}")
print(f"Rows marked as 'Yes': {len(matched_indices)}")
print(f"Rows marked as 'No': {len(Fresh_Leads_formatted) - len(matched_indices)}")

# Create a summary DataFrame
if matches_found:
    matches_df = pd.DataFrame(matches_found)
    print("\nSummary of matches:")
    print(matches_df)
else:
    print("\nNo matches found above the threshold.")

# Display the updated dataframe with the new column
print("\n\nUpdated Fresh_Leads_formatted with 'Current ePOS Client ?' column:")
print(Fresh_Leads_formatted[['ACRA REGISTERED NAME', 'Current ePOS Client ?']].head(20))


Checking 368 names against 2343 ePOS backend organizations...



Total matches found: 0 out of 368
Rows marked as 'Yes': 0
Rows marked as 'No': 368

No matches found above the threshold.


Updated Fresh_Leads_formatted with 'Current ePOS Client ?' column:
                                ACRA REGISTERED NAME Current ePOS Client ?
0                         BELL 1 SINGAPORE PTE. LTD.                    No
1                                  NUU&NII PTE. LTD.                    No
2                      CLOUD FRONTIER ASIA PTE. LTD.                    No
3             GOLDEN GARLAND COMMUNICATION PTE. LTD.                    No
4                            XINYUAN SHOES PTE. LTD.                    No
5                         THESEUS PROMOTER PTE. LTD.                    No
6                    ASIA DIGITAL ALLIANCE PTE. LTD.                    No
7                         NINE PURPLE GOLD PTE. LTD.                    No
8                         MYFIT ENTERPRISE PTE. LTD.                 

In [64]:
Fresh_Leads_formatted

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,None,None,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
1,None,None,2025-11-10,NUU&NII PTE. LTD.,NUU&NII,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
2,None,None,2025-11-10,CLOUD FRONTIER ASIA PTE. LTD.,CLOUD FRONTIER ASIA,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
3,None,None,2025-11-10,GOLDEN GARLAND COMMUNICATION PTE. LTD.,GOLDEN GARLAND COMMUNICATION,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
4,None,None,2025-11-10,XINYUAN SHOES PTE. LTD.,XINYUAN SHOES,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,None,None,2025-11-10,RSZB INTERNATIONAL DEVELOPMENT PTE. LTD.,RSZB INTERNATIONAL DEVELOPMENT,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
364,None,None,2025-11-10,A&K MARINE SUPPLIES PTE. LTD.,A&K MARINE SUPPLIES,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
365,None,None,2025-11-10,KAPI KAPI BY LUVI PTE. LTD.,KAPI KAPI BY LUVI,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
366,None,None,2025-11-10,J ELITE FOOD,J ELITE FOOD,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None


### Filtering with Master DB

In [65]:

# --- CONFIG ---
file_path = "./Master DB/Master_DB_nov10.xlsx"

# --- HELPER FUNCTIONS ---
def clean_uen(u: str) -> str | None:
    if pd.isna(u):
        return None
    return re.sub(r"[^A-Z0-9]", "", str(u).upper().strip())

def clean_text(text: str) -> str | None:
    if pd.isna(text):
        return None
    text = str(text).strip().upper()
    return None if text == "NAN" else text

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert all column names to uppercase, replace non-alphanumeric with single underscore, remove trailing underscores."""
    new_cols = []
    for col in df.columns:
        col_std = re.sub(r"[^A-Z0-9]", "_", col.upper().strip())
        col_std = re.sub(r"_+", "_", col_std)  # Replace multiple underscores with single
        col_std = col_std.strip("_")  # Remove leading/trailing underscores
        new_cols.append(col_std)
    df.columns = new_cols
    return df

# --- LOAD DATA ---
master_db_df = pd.read_excel(file_path)

# --- SELECT RELEVANT COLUMNS ---
columns_to_keep = [
    "Company Registration Number (UEN)",
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name",
    "Primary SSIC Code",
    "PIC NAME 1 Contact Number",
    "PIC 1 email address",
    "Website URL",
    "Parent Industry Type",
    "Sub Industry"
]
master_db_df = master_db_df[columns_to_keep].copy()

# --- STANDARDIZE COLUMN NAMES ---
master_db_df = standardize_columns(master_db_df)

# --- CLEANING & RENAME SPECIFIC COLUMNS ---
# Dynamically find the UEN column (first column containing 'UEN')
uen_col = [c for c in master_db_df.columns if "UEN" in c][0]
master_db_df["UEN"] = master_db_df[uen_col].apply(clean_uen)
master_db_df = master_db_df.drop(columns=[uen_col])

# Rename other columns consistently
rename_map = {
    "BRAND_DEAL_NAME_BUSINESS_NAME": "BRAND_NAME",
    "PRIMARY_SSIC_CODE": "SSIC_CODE",
    "ACRA_REGISTERED_NAME": "ACRA_REGISTERED_NAME"
}
master_db_df = master_db_df.rename(columns={k: v for k, v in rename_map.items() if k in master_db_df.columns})

# Clean text columns
for col in ["ACRA_REGISTERED_NAME", "BRAND_NAME"]:
    if col in master_db_df.columns:
        master_db_df[col] = master_db_df[col].apply(clean_text)

# Convert SSIC_CODE to integer if exists
if "SSIC_CODE" in master_db_df.columns:
    master_db_df["SSIC_CODE"] = master_db_df["SSIC_CODE"].astype("Int64")

# Keep only required columns if they exist
required_cols = ["UEN", "ACRA_REGISTERED_NAME", "BRAND_NAME", "SSIC_CODE"]
master_db_df = master_db_df[[c for c in required_cols if c in master_db_df.columns]]

# Filter out rows with missing or empty UEN
master_db_df = master_db_df[master_db_df["UEN"].notna() & (master_db_df["UEN"].str.strip() != "")]

master_db_df


,UEN,ACRA_REGISTERED_NAME,BRAND_NAME,SSIC_CODE
0,04799400B,AIK BEE TEXTILE CO,AIK BEE TEXTILE CO,46411
1,03376200K,SERANGOON GARDEN CLINIC AND DISPENSARY,GARDEN CLINIC,550263
2,06239600E,SALON DE BENZIMEN,SALON DE BENZIMEN,96021
3,06952000C,SU LAN LADIES FASHION,SU LAN LADIES FASHION,14103
4,10381600C,SIN HAI PRINTING SERVICE,SIN HAI PRINTING SERVICE,18113
...,...,...,...,...
10116,202306640R,MISTER MOBILE POTONG PASIR PTE. LTD.,MISTER MOBILE (POTONG PASIR),47411
10117,202103010N,MISTER MOBILE JURONG PTE. LTD.,MISTER MOBILE (JURONG),47411
10118,201734006N,MISTER MOBILE HOUGANG PTE. LTD.,MISTER MOBILE (HOUGANG),95120
10119,202210879W,MISTER MOBILE CHINATOWN PTE. LTD.,MISTER MOBILE (CHINATOWN),47411


In [66]:
# Compare UEN values between master_db_df and Fresh_Leads_formatted

# Clean UEN values from Fresh_Leads_formatted using the same cleaning function
Fresh_Leads_formatted['UEN_cleaned'] = Fresh_Leads_formatted['Company Registration Number (UEN)'].apply(clean_uen)

# Get valid UENs from both dataframes
master_uen_set = set(master_db_df['UEN'].dropna().astype(str))
fresh_leads_uen_series = Fresh_Leads_formatted['UEN_cleaned'].dropna().astype(str)

# Find matches
matches = []
for idx, uen in fresh_leads_uen_series.items():
    if uen in master_uen_set:
        matches.append({
            'Index': idx,
            'UEN': uen,
            'ACRA REGISTERED NAME': Fresh_Leads_formatted.at[idx, 'ACRA REGISTERED NAME'],
            'Brand/Deal Name': Fresh_Leads_formatted.at[idx, 'Brand/Deal Name/Business Name']
        })
        print(f"Match found! UEN: {uen}")
        print(f"  ACRA Name: {Fresh_Leads_formatted.at[idx, 'ACRA REGISTERED NAME']}")
        print(f"  Brand Name: {Fresh_Leads_formatted.at[idx, 'Brand/Deal Name/Business Name']}")
        print("-" * 80)

print(f"\n\nTotal matches found: {len(matches)} out of {len(fresh_leads_uen_series)} UENs in Fresh_Leads_formatted")
print(f"Master DB contains {len(master_uen_set)} unique UENs")

# Create summary DataFrame
if matches:
    matches_df = pd.DataFrame(matches)
    print("\nSummary of UEN matches:")
    print(matches_df)
else:
    print("\nNo UEN matches found between master_db_df and Fresh_Leads_formatted.")

# Drop the temporary cleaned column
Fresh_Leads_formatted = Fresh_Leads_formatted.drop(columns=['UEN_cleaned'])




Total matches found: 0 out of 368 UENs in Fresh_Leads_formatted
Master DB contains 5880 unique UENs

No UEN matches found between master_db_df and Fresh_Leads_formatted.


In [67]:
Fresh_Leads_formatted

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,None,None,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
1,None,None,2025-11-10,NUU&NII PTE. LTD.,NUU&NII,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
2,None,None,2025-11-10,CLOUD FRONTIER ASIA PTE. LTD.,CLOUD FRONTIER ASIA,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
3,None,None,2025-11-10,GOLDEN GARLAND COMMUNICATION PTE. LTD.,GOLDEN GARLAND COMMUNICATION,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
4,None,None,2025-11-10,XINYUAN SHOES PTE. LTD.,XINYUAN SHOES,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,None,None,2025-11-10,RSZB INTERNATIONAL DEVELOPMENT PTE. LTD.,RSZB INTERNATIONAL DEVELOPMENT,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
364,None,None,2025-11-10,A&K MARINE SUPPLIES PTE. LTD.,A&K MARINE SUPPLIES,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
365,None,None,2025-11-10,KAPI KAPI BY LUVI PTE. LTD.,KAPI KAPI BY LUVI,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
366,None,None,2025-11-10,J ELITE FOOD,J ELITE FOOD,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None


In [68]:
# Filter Fresh_Leads_formatted to get rows where "PIC NAME 1 Contact Number" has a value (not NaN)
Fresh_Leads_Nov11 = Fresh_Leads_formatted[
    Fresh_Leads_formatted['PIC NAME 1 Contact Number'].notna()
].copy()

print(f"Original Fresh_Leads_formatted rows: {len(Fresh_Leads_formatted)}")
print(f"Filtered Fresh_Leads_Nov11 rows (with PIC NAME 1 Contact Number): {len(Fresh_Leads_Nov11)}")
print(f"Rows removed (NaN in PIC NAME 1 Contact Number): {len(Fresh_Leads_formatted) - len(Fresh_Leads_Nov11)}")

# Display the filtered dataframe
print("\n\nFresh_Leads_Nov11 (first 10 rows):")
Fresh_Leads_Nov11.head(10)


Original Fresh_Leads_formatted rows: 368
Filtered Fresh_Leads_Nov11 rows (with PIC NAME 1 Contact Number): 105
Rows removed (NaN in PIC NAME 1 Contact Number): 263


Fresh_Leads_Nov11 (first 10 rows):


,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,None,None,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
5,None,None,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
8,None,None,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
13,None,None,2025-11-10,THE SINGAPORE FOOD AND WINE,THE FOOD AND WINE,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
17,None,None,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
18,None,None,2025-11-10,ABBACI DISTRIBUTION (SINGAPORE) PTE. LTD.,ABBACI DISTRIBUTION,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
22,None,None,2025-11-10,PEM (SG) PTE. LTD.,PEM SG,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
24,None,None,2025-11-10,312 TECHNOLOGY LLP,312 TECHNOLOGY,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
25,None,None,2025-11-10,SUSTAIN ENGINEERING PTE. LTD.,SUSTAIN ENGINEERING,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None
27,None,None,2025-11-10,SLD SEMI (SINGAPORE) PTE. LTD.,SLD SEMI,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,"[ACRA, Google Searches]",No,None


In [69]:
# Remove [ ], ', +, and , (comma) from specified columns in Fresh_Leads_Nov11
columns_to_clean = [
    'PIC NAME 1 Contact Number',
    'PIC 1 email address',
    'Source from Market Researcher',
    'Business model',
    'Facebook Page',
    'Instagram URL',
    'Linkedin URL',
    'Tik Tok URL'
]

# Characters to remove: [ ] ' + ,
chars_to_remove = ['[', ']', "'", '+', ',']

# Clean each column by removing specified characters
for col in columns_to_clean:
    if col in Fresh_Leads_Nov11.columns:
        # Convert to string and remove all specified characters
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].astype(str)
        for char in chars_to_remove:
            Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].str.replace(char, '', regex=False)
        # Strip any extra whitespace
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].str.strip()
        # Replace 'nan' string back to actual NaN for empty values
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].replace('nan', pd.NA).replace('', pd.NA)
        print(f"Cleaned column: {col}")
    else:
        print(f"Warning: Column '{col}' not found in dataframe")

print(f"\nCharacters removed ([ ] ' + ,) from {len([c for c in columns_to_clean if c in Fresh_Leads_Nov11.columns])} columns")

# Function to format phone numbers to "65 XXXXXXXX" format with spaces
def format_phone_number(phone_str):
    """
    Format phone number to "65 XXXXXXXX" format with spaces.
    Extracts all digits and formats as: 65 [4 digits] [4 digits]
    """
    if pd.isna(phone_str) or phone_str == '' or phone_str == 'nan':
        return pd.NA
    
    # Extract all digits from the string
    digits_only = re.sub(r'\D', '', str(phone_str))
    
    # If empty after extracting digits, return NA
    if not digits_only:
        return pd.NA
    
    # Handle different formats
    # If starts with 65, format as "65 XXXX XXXX" (8 digits after 65)
    if digits_only.startswith('65') and len(digits_only) >= 10:
        # Remove the 65 prefix and get the remaining digits
        remaining = digits_only[2:]
        # Format as "65 XXXX XXXX" (4 digits, space, 4 digits)
        if len(remaining) >= 8:
            formatted = f"65 {remaining[:4]} {remaining[4:8]}"
            # If there are more digits, append them
            if len(remaining) > 8:
                formatted += f" {remaining[8:]}"
            return formatted
        else:
            # Less than 8 digits after 65, format as "65 XXXXXX"
            return f"65 {remaining}"
    
    # If doesn't start with 65 but has 8+ digits, assume it's a local number and add 65
    elif len(digits_only) >= 8:
        # Format as "65 XXXX XXXX"
        formatted = f"65 {digits_only[:4]} {digits_only[4:8]}"
        if len(digits_only) > 8:
            formatted += f" {digits_only[8:]}"
        return formatted
    
    # If less than 8 digits, just return as is (might be extension or incomplete)
    else:
        return digits_only

# Apply phone number formatting to "PIC NAME 1 Contact Number" column
if 'PIC NAME 1 Contact Number' in Fresh_Leads_Nov11.columns:
    Fresh_Leads_Nov11['PIC NAME 1 Contact Number'] = Fresh_Leads_Nov11['PIC NAME 1 Contact Number'].apply(format_phone_number)
    print("\nPhone numbers formatted to '65 XXXX XXXX' format")

print("\nSample of cleaned data:")
# Only show columns that exist in the dataframe
existing_columns = [col for col in columns_to_clean if col in Fresh_Leads_Nov11.columns]
if existing_columns:
    Fresh_Leads_Nov11[existing_columns].head(10)
else:
    print("No columns to display")


Cleaned column: PIC NAME 1 Contact Number
Cleaned column: PIC 1 email address
Cleaned column: Source from Market Researcher
Cleaned column: Business model
Cleaned column: Facebook Page
Cleaned column: Instagram URL
Cleaned column: Linkedin URL
Cleaned column: Tik Tok URL

Characters removed ([ ] ' + ,) from 8 columns

Phone numbers formatted to '65 XXXX XXXX' format

Sample of cleaned data:


In [70]:
Fresh_Leads_Nov11.head()

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,None,None,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,ACRA Google Searches,No,None
5,None,None,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,ACRA Google Searches,No,None
8,None,None,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,ACRA Google Searches,No,None
13,None,None,2025-11-10,THE SINGAPORE FOOD AND WINE,THE FOOD AND WINE,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,ACRA Google Searches,No,None
17,None,None,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE,None,TeleSales,Kalaivani,Offline Sources,Web Scrapping,...,None,None,None,None,No,None,Gov List,ACRA Google Searches,No,None


In [ ]:
Fresh_Leads_Nov11.to_csv("Golden_Data/Fresh_Leads_with_PhoneNumber_Nov11.csv", index=False)